In [0]:
%run
./display_images

In [0]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import pickle
import datetime
import pytz 
from numpy.random import choice
from google.cloud import storage
import io
import lightning.pytorch as pl
import torch.nn.functional as F
import faiss


storage_client = storage.Client()
bucket = storage_client.get_bucket('gcs-dsci-fryou-fy-dev-prd')

/databricks/python/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/databricks/python3/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/databricks/python/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/databricks/python/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pyt

# Generate Data

In [0]:
path = "gs://gcs-dsci-fryou-fy-dev-prd/catalog_vqvae/vqvae_catalog_embeddings_2025-03-01"
catalog_embedding_1m = spark.read.parquet(path)
version = "v4"

In [0]:
pdf_catalog_embedding = catalog_embedding_1m.limit(100000).toPandas()

In [0]:
# chunk_size = 100000 
# num_chunks = (catalog_embedding_1m.count() // chunk_size) + 1

# pdf_list = []
# for i in range(num_chunks):
#     chunk = catalog_embedding_1m.limit(chunk_size).toPandas()
#     pdf_list.append(chunk)

# pdf_catalog_embedding = pd.concat(pdf_list, ignore_index=True)


In [0]:
catalog_id_index = dict(zip(pdf_catalog_embedding.catalog_id.values,np.arange(len(pdf_catalog_embedding))))

In [0]:
index_to_catalog_id = dict(zip(catalog_id_index.values(), catalog_id_index.keys()))

In [0]:
# pdf_catalog_embedding = catalog_embedding_1m.toPandas()
pdf_catalog_embedding.head()

,catalog_id,search_catalog_embedding
0,3411,"[-0.030754955, -0.1513503, -0.025996726, -0.05..."
1,34937,"[-0.12764944, 0.07539975, 0.059684392, 0.05368..."
2,37310,"[-0.027382225, -0.030700022, 0.08685166, 0.021..."
3,42268,"[0.10540673, 0.007949251, -0.10770779, 0.14287..."
4,63271,"[0.084498696, 0.04375279, -0.03983832, -0.0375..."


In [0]:
catalog_embeddings = pdf_catalog_embedding.search_catalog_embedding.to_list()
catalog_embeddings = np.array(catalog_embeddings, dtype=np.float32)

In [0]:
t_catalog_embeddings = torch.tensor(catalog_embeddings)

In [0]:
version = "v4"

In [0]:
en_model_blob = bucket.get_blob(f'catalog_vqvae/checkpoint_dir/{version}/epoch=1-step=2724.ckpt')
en_model = en_model_blob.download_as_string()


# because model downloaded into string, need to convert it back
buffer = io.BytesIO(en_model)
state_dict = torch.load(buffer, map_location=torch.device('cpu'))

In [0]:
class Encoder(nn.Module):
    def __init__(self, input_size, output_size): 
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size

        self.encoder = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(128, 128),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(128, output_size)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = x/(x.norm(dim=-1, keepdim=True).detach())
        return x

In [0]:
batch_size = 5120
num_workers = 8
input_size = 128 
output_size =16
num_embeddings = 10000
embedding_dim = 16 
commitment_cost = 20

In [0]:
model_weights = state_dict["state_dict"]
final_weights = {}

# update keys by dropping `auto_encoder.`
for key in list(model_weights):
    if "model.enc" in key:
        final_weights[key.replace("model.enc.", "", 1)] = model_weights[key]

enc = Encoder(input_size, output_size)
enc.load_state_dict(final_weights)
enc.eval()

Encoder(
  (encoder): Sequential(
    (0): Linear(in_features=128, out_features=128, bias=True)
    (1): GELU(approximate='none')
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=128, out_features=128, bias=True)
    (4): GELU(approximate='none')
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=128, out_features=16, bias=True)
  )
)

In [0]:
enc_output = enc(t_catalog_embeddings)

In [0]:
encoder_embeddings = enc_output.detach().numpy()

In [0]:
encoder_embeddings.shape

(100000, 16)

In [0]:
num_to_zero = int(0.1 * 100000)

indices_to_zero = np.random.choice(100000, num_to_zero, replace=False)
# print(indices_to_zero)

encoder_embeddings[indices_to_zero] = 0

display(encoder_embeddings)

array([[ 0.1009838 , -0.1833219 , -0.31495994, ..., -0.23286115,
        -0.4414628 , -0.19689545],
       [-0.09266586,  0.10836744, -0.42532012, ..., -0.0244863 ,
         0.03175486, -0.23987392],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.00139187, -0.04974828, -0.22377451, ...,  0.534535  ,
         0.5021528 , -0.30573496],
       [ 0.17449637,  0.342192  , -0.20039952, ..., -0.02746953,
        -0.32018235,  0.2422805 ],
       [-0.22886105, -0.0484769 , -0.24758136, ...,  0.01165675,
        -0.4167883 ,  0.00367156]], dtype=float32)

In [0]:
import faiss

index = faiss.IndexFlatL2(16)
faiss.normalize_L2(encoder_embeddings)
index.add(encoder_embeddings)

In [0]:
print(encoder_embeddings.shape)  # Should output (128,)

(100000, 16)


In [0]:
def get_similar_catalog(id, embeddings, index, k=20):
    res = index.search(np.array([embeddings[id]]), k)
    return [index_to_catalog_id[ele] for ele in res[1][0]]

In [0]:
id = 56
catalogs= get_similar_catalog(id, encoder_embeddings, index, k=200)
display_images(catalogs)

Gardening 
 Live Plants 
 45 
 
 
 
 
 Puja Accessorie... 
 Puja Articles 
 2 
 
 
 
 
 Women Inner & S... 
 Nightdress 
 1 
 
 
 
 
 Hair Care 
 Mehendi 
 9 
 
 
 
 
 Women Inner & S... 
 Nightdress 
 4 
 
 
 
 
 Women Inner & S... 
 Nightdress 
 2 
 
 
 
 
 Puja Accessorie... 
 Puja Articles 
 8 
 
 
 
 
 Women Inner & S... 
 Nightdress 
 2 
 
 
 
 
 Puja Accessorie... 
 Puja Articles 
 4 
 
 
 
 
 Women Inner & S... 
 Nightdress 
 1 
 
 
 
 
 Hair Care 
 Mehendi 
 26 
 
 
 
 
 Puja Accessorie... 
 Puja Articles 
 4 
 
 
 
 
 Women Inner & S... 
 Nightdress 
 21 
 
 
 
 
 Women Accessori... 
 Bindis 
 25 
 
 
 
 
 Women Inner & S... 
 Nightdress 
 23 
 
 
 
 
 Women Inner & S... 
 Nightdress 
 22 
 
 
 
 
 Women Inner & S... 
 Nightdress 
 19 
 
 
 
 
 Women Jewellery 
 Mangalsutras 
 11 
 
 
 
 
 Women Inner & S... 
 Nightdress 
 19 
 
 
 
 
 Women Inner & S... 
 Nightdress 
 43 
 
 
 
 
 Women Ethnic We... 
 Dupattas 
 33 
 
 
 
 
 Light Bulbs 
 Light Bulbs 
 19 
 
 
 
 
 Women Inner & S... 
 Nightdress 
 31 
 
 
 
 
 Women Jewellery 
 Mangalsutras 
 23 
 
 
 
 
 Puja Accessorie... 
 Puja Articles 
 18 
 
 
 
 
 Women Inner & S... 
 Nightdress 
 2 
 
 
 
 
 Women Inner & S... 
 Nightdress 
 16 
 
 
 
 
 Women Inner & S... 
 Nightdress 
 1 
 
 
 
 
 Puja Accessorie... 
 Puja Articles 
 13 
 
 
 
 
 Puja Accessorie... 
 Puja Articles 
 11 
 
 
 
 
 Women Inner & S... 
 Nightdress 
 1 
 
 
 
 
 Women Inner & S... 
 Nightdress 
 14 
 
 
 
 
 Puja Accessorie... 
 Puja Articles 
 5 
 
 
 
 
 Puja Accessorie... 
 Puja Articles 
 10 
 
 
 
 
 Women Accessori... 
 Bindis 
 21 
 
 
 
 
 Women Accessori... 
 Bindis 
 28 
 
 
 
 
 Hair Styling & ... 
 Hair Color 
 12 
 
 
 
 
 Gardening 
 Live Plants 
 12 
 
 
 
 
 Puja Accessorie... 
 Puja Articles 
 31 
 
 
 
 
 Women Ethnic We... 
 Dupattas 
 29 
 
 
 
 
 Gardening 
 Seeds 
 3 
 
 
 
 
 Puja Accessorie... 
 Puja Articles 
 25 
 
 
 
 
 Puja Accessorie... 
 Puja Articles 
 46 
 
 
 
 
 Women Jewellery 
 Nosepins 
 39 
 
 
 
 
 Women Inner & S... 
 Nightdress 
 2 
 
 
 
 
 Puja Accessorie... 
 Puja Articles 
 25 
 
 
 
 
 Women Inner & S... 
 Nightdress 
 7 
 
 
 
 
 Puja Accessorie... 
 Puja Articles 
 4 
 
 
 
 
 Women Inner & S... 
 Nightdress 
 3 
 
 
 
 
 Puja Accessorie... 
 Puja Articles 
 13 
 
 
 
 
 Ayurveda 
 Ayurvedic Skin ... 
 19 
 
 
 
 
 Women Accessori... 
 Stoles 
 16 
 
 
 
 
 Women Accessori... 
 Bindis 
 17 
 
 
 
 
 Decorative Acce... 
 Rangoli 
 33 
 
 
 
 
 Women Jewellery 
 Maangtika 
 13 
 
 
 
 
 Puja Accessorie... 
 Puja Articles 
 9 
 
 
 
 
 Puja Accessorie... 
 Puja Articles 
 32 
 
 
 
 
 Women Inner & S... 
 Nightdress 
 6 
 
 
 
 
 Women Ethnic We... 
 Dupattas 
 39 
 
 
 
 
 Women Jewellery 
 Jewellery Set 
 7 
 
 
 
 
 Decorative Acce... 
 Rangoli 
 19 
 
 
 
 
 Puja Accessorie... 
 Puja Articles 
 4 
 
 
 
 
 Puja Accessorie... 
 Puja Articles 
 8 
 
 
 
 
 Puja Accessorie... 
 Puja Articles 
 7 
 
 
 
 
 Pens, Pencils &... 
 Chalk 
 35 
 
 
 
 
 Hair Care 
 Hair Oil 
 29 
 
 
 
 
 Puja Accessorie... 
 Puja Articles 
 5 
 
 
 
 
 Gardening 
 Seeds 
 2 
 
 
 
 
 Snacks & Namkee... 
 Chips 
 11 
 
 
 
 
 Puja Accessorie... 
 Puja Articles 
 6 
 
 
 
 
 Women Ethnic We... 
 Dupattas 
 38 
 
 
 
 
 Gardening 
 Seeds 
 7 
 
 
 
 
 Women Inner & S... 
 Nightdress 
 3 
 
 
 
 
 Gardening 
 Live Plants 
 21 
 
 
 
 
 Women Inner & S... 
 Nightdress 
 21 
 
 
 
 
 Puja Accessorie... 
 Puja Articles 
 15 
 
 
 
 
 Decorative Acce... 
 Rangoli 
 45 
 
 
 
 
 Puja Accessorie... 
 Puja Articles 
 3 
 
 
 
 
 Women Jewellery 
 Jewellery Set 
 10 
 
 
 
 
 Puja Accessorie... 
 Puja Articles 
 16 
 
 
 
 
 Women Jewellery 
 Maangtika 
 26 
 
 
 
 
 Women Inner & S... 
 Nightdress 
 1 
 
 
 
 
 Women Inner & S... 
 Nightdress 
 18 
 
 
 
 
 Women Inner & S... 
 Nightdress 
 3 
 
 
 
 
 Home Medicines 
 Ayurvedic Produ... 
 17 
 
 
 
 
 Women Inner & S... 
 Nightdress 
 11 
 
 
 
 
 Hair Care 
 Hair Packs & Po... 
 19 
 
 
 
 
 Women Inner & S... 
 Nightdress 
 10 

In [0]:
import random

id = random.randint(0, 100000)
print(id)
catalogs = get_similar_catalog(id, encoder_embeddings, index, k=200)
display_images(catalogs)

56033


Kids Accessorie... 
 Belts 
 32 
 
 
 
 
 Kids - Boys Wes... 
 Shorts & Capris 
 28 
 
 
 
 
 Girls Innerwear 
 Girls Innerwear 
 9 
 
 
 
 
 Baby Personal C... 
 Baby Cloth Diap... 
 41 
 
 
 
 
 Kids - Girls We... 
 Shorts & Capris 
 39 
 
 
 
 
 Kids - Boys Wes... 
 Shorts & Capris 
 23 
 
 
 
 
 Baby Care 
 Baby Daipers 
 23 
 
 
 
 
 Boys Sleepwear 
 Pyjamas 
 30 
 
 
 
 
 Kids - Boys Wes... 
 Nightsuits 
 9 
 
 
 
 
 Kids - Boys Wes... 
 Innerwear 
 23 
 
 
 
 
 Kids - Boys Wes... 
 Innerwear 
 29 
 
 
 
 
 Kids Accessorie... 
 Socks 
 8 
 
 
 
 
 Accessories 
 Hats & Caps 
 18 
 
 
 
 
 Girls Sleepwear 
 Pyjamas 
 3 
 
 
 
 
 Kids - Boys Wes... 
 Sweaters 
 1 
 
 
 
 
 Baby & Nursery ... 
 Pillows & Pillo... 
 16 
 
 
 
 
 Baby Care 
 Baby Towels 
 35 
 
 
 
 
 Kids Accessorie... 
 Socks 
 7 
 
 
 
 
 Kids Accessorie... 
 Belts 
 32 
 
 
 
 
 Baby Personal C... 
 Baby Cloth Diap... 
 39 
 
 
 
 
 Kids - Boys Wes... 
 Innerwear 
 2 
 
 
 
 
 Kids - Girls We... 
 Shorts & Capris 
 4 
 
 
 
 
 Baby Care 
 Baby Wipes 
 16 
 
 
 
 
 Girls Innerwear 
 Girls Innerwear 
 2 
 
 
 
 
 Kids - Girls We... 
 Shorts & Capris 
 39 
 
 
 
 
 Kids - Boys Wes... 
 Innerwear 
 25 
 
 
 
 
 Winterwear 
 Boys Earmuffs 
 13 
 
 
 
 
 Baby Care 
 Baby Wipes 
 10 
 
 
 
 
 Accessories 
 Hats & Caps 
 37 
 
 
 
 
 Kids - Boys Wes... 
 Innerwear 
 16 
 
 
 
 
 Baby Care 
 Baby Wipes 
 41 
 
 
 
 
 Winterwear 
 Boys Gloves 
 8 
 
 
 
 
 Orthopedic supp... 
 Orthopedic Knee... 
 23 
 
 
 
 
 Kids Accessorie... 
 Belts 
 20 
 
 
 
 
 Kids Accessorie... 
 Belts 
 27 
 
 
 
 
 Baby Care 
 Baby Blanket 
 3 
 
 
 
 
 Baby Personal C... 
 Baby Cloth Diap... 
 16 
 
 
 
 
 Baby Personal C... 
 Baby Cloth Diap... 
 21 
 
 
 
 
 Baby Care 
 Baby Oil 
 44 
 
 
 
 
 Accessories 
 Hats & Caps 
 5 
 
 
 
 
 Kids - Boys Wes... 
 Shorts & Capris 
 3 
 
 
 
 
 Kids Accessorie... 
 Caps 
 28 
 
 
 
 
 Kids - Boys Wes... 
 Innerwear 
 29 
 
 
 
 
 Kids Accessorie... 
 Belts 
 1 
 
 
 
 
 Kids - Boys Wes... 
 Innerwear 
 24 
 
 
 
 
 Baby & Nursery ... 
 Pillows & Pillo... 
 6 
 
 
 
 
 Kids Accessorie... 
 Caps 
 32 
 
 
 
 
 Orthopedic supp... 
 Orthopedic Knee... 
 7 
 
 
 
 
 Kids Accessorie... 
 Belts 
 7 
 
 
 
 
 Kids - Boys Wes... 
 Shorts & Capris 
 18 
 
 
 
 
 Winterwear 
 Boys Gloves 
 18 
 
 
 
 
 Kids - Boys Wes... 
 Innerwear 
 15 
 
 
 
 
 Kids - Girls We... 
 Shorts & Capris 
 21 
 
 
 
 
 Baby Personal C... 
 Baby Cloth Diap... 
 4 
 
 
 
 
 Infants 
 Booties 
 18 
 
 
 
 
 Kids - Boys Wes... 
 Nightsuits 
 21 
 
 
 
 
 Kids - Boys Wes... 
 Innerwear 
 1 
 
 
 
 
 Kitchen Linens 
 Aprons 
 2 
 
 
 
 
 Kids - Girls We... 
 Shorts & Capris 
 28 
 
 
 
 
 Kids - Boys Wes... 
 Shorts & Capris 
 40 
 
 
 
 
 Kids - Girls We... 
 Oneseis & Rompe... 
 18 
 
 
 
 
 Kids Accessorie... 
 Belts 
 23 
 
 
 
 
 Kids - Boys Wes... 
 Innerwear 
 8 
 
 
 
 
 Kids - Girls We... 
 Sweaters 
 45 
 
 
 
 
 Baby Care 
 Baby Daipers 
 2 
 
 
 
 
 Kids Accessorie... 
 Caps 
 24 
 
 
 
 
 Baby Personal C... 
 Baby Cloth Diap... 
 20 
 
 
 
 
 Kids - Girls We... 
 Thermals 
 2 
 
 
 
 
 Kids - Boys Wes... 
 Innerwear 
 20 
 
 
 
 
 Kids Accessorie... 
 Belts 
 27 
 
 
 
 
 Kids - Boys Wes... 
 Nightsuits 
 37 
 
 
 
 
 Kids Accessorie... 
 Caps 
 36 
 
 
 
 
 Kids - Girls We... 
 Shorts & Capris 
 10 
 
 
 
 
 Kids - Boys Wes... 
 Innerwear 
 5 
 
 
 
 
 Kids Accessorie... 
 Socks 
 36 
 
 
 
 
 Kids - Boys Wes... 
 Shorts & Capris 
 10 
 
 
 
 
 Arts & Crafts 
 Stickers 
 12 
 
 
 
 
 Accessories 
 Hats & Caps 
 46 
 
 
 
 
 Kids Accessorie... 
 Caps 
 41 
 
 
 
 
 Baby Care 
 Baby Towels 
 12 
 
 
 
 
 Baby Personal C... 
 Baby Cloth Diap... 
 4 
 
 
 
 
 Accessories 
 Hats & Caps 
 32 
 
 
 
 
 Accessories 
 Hats & Caps 
 5 
 
 
 
 
 Kids - Girls We... 
 Shorts & Capris 
 9 
 
 
 
 
 Kids Accessorie... 
 Socks 
 21 
 
 
 
 
 Girls Innerwear 
 Girls Innerwear 
 1 
 
 
 
 
 Accessories 
 Hats & Caps 
 15 
 
 
 
 
 Winterwear 
 Boys Gloves 
 24 
 
 
 
 
 Kids Accessorie...

In [0]:
notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
notebook_path


'/Users/arshi.khan@meesho.com/catalog_embeddings'